1. 데이터셋의 차원을 축소하는 목적은 무엇인가요? 대표적인 단점은 무엇인가요?

목적은 모델 훈련 속도가 빨라진다는 점이고, 단점은 축소하는 과정에서 정보유실이 발생할 수 있다는 것이다. 

2. 기본 PCA, 점진적 PCA, 랜덤 PCA,커널 PCA는 어느 경우에 사용될까요?

기본 PCA : 차원 축소할 때 사용함
점진적 PCA: 데이터 셋의 크기가 클 때 사용함. 
랜덤 PCA: 차원을 80%이하로 줄일 때 사용함. 
커널 PCA: 투영된 후 샘플의 군집을 유지하거나 꼬인 매니폴드를 펼칠 때 유용함. 

3. MNIST 데이터셋을 로드하고 훈련세트와 테스트세트로 분할하겠습니다.

# 모듈 설정 및 데이터 불러오기

In [1]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# MNIST 불러오기
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.target = mnist.target.astype(np.uint8)

# 훈련세트와 테스트세트로 분할

In [18]:
# 첫 60000개는 훈련을 위한 샘플, 나머지 10000개는 테스트용으로 분할
X_train = mnist['data'][:60000]
y_train = mnist['target'][:60000]

X_test = mnist['data'][60000:]
y_test = mnist['target'][60000:]

3.1 이 데이터셋을 랜덤 포레스트 분류기로 훈련시키고 시간이 얼마나 오래걸리는지 시간을 잰 후, 테스트 세트로 만들어진 모델의 정확도를 평가합니다.

In [3]:
# 랜덤포레스트로 훈련시키는 코드
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)

In [4]:
# 훈련 시간을 재는 코드입니다.
import time

t0 = time.time()
rnd_clf.fit(X_train, y_train)
t1 = time.time()

print("훈련 시간: {:.2f}s".format(t1 - t0))

훈련 시간: 168.25s


In [7]:
# 정확도 평가 코드입니다.
from sklearn.metrics import accuracy_score

y_pred = rnd_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9705

**3.2 문제 : 데이터셋을 PCA를 사용해 설명된 분산이 95%가 되도록 차원을 축소하세요**

In [6]:
from sklearn.decomposition import PCA
pca=PCA(n_components=0.95)
pca.fit_transform(X_train)

array([[ 123.93258866, -312.67426202,  -24.51405176, ...,   55.01899792,
         -20.08327427,   39.58995229],
       [1011.71837587, -294.85703827,  596.33956104, ...,    7.24129874,
         -12.45780869,  -12.7432306 ],
       [ -51.84960805,  392.17315286, -188.50974943, ...,  -54.19582221,
          48.47979747,  -73.27826256],
       ...,
       [-178.0534496 ,  160.07821109, -257.61308227, ...,   55.54485537,
          87.99883556,   -5.78979735],
       [ 130.60607208,   -5.59193642,  513.85867395, ...,   23.30835402,
           5.06237836,  -65.26525587],
       [-173.43595244,  -24.71880226,  556.01889393, ...,   52.4956069 ,
          12.63192292,  -45.74001227]])

**3.3 문제 : 이 축소된 데이터셋에 위와 같이 새로운 랜덤포레스트 분류기를 훈련시키고 얼마나 오래걸리는지 확인합니다. 훈련 시간이 빨라졌나요?**

In [7]:
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)


t0 = time.time()
rnd_clf.fit(X_train, y_train)
t1 = time.time()

print("훈련 시간: {:.2f}s".format(t1 - t0))

훈련 시간: 162.08s


훈련시간이 빨라짐

**3.4 문제: 이 테스트 세트에서 이 분류기의 정확도를 평가해보세요. 이전 분류기와 비교해서 어떤가요?**

In [9]:
from sklearn.metrics import accuracy_score
y_pred = rnd_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9705

이전 분류기와 정확도가 똑같음

3.5 소프트 맥스 회귀를 사용해서도 같은 방식으로 실험을 해봅니다.

In [19]:
# 차원축소 이전의 코드
from sklearn.linear_model import LogisticRegression

log_clf = LogisticRegression(multi_class="multinomial", solver="lbfgs", random_state=42)
t0 = time.time()
log_clf.fit(X_train, y_train)
t1 = time.time()

C:\Users\jeehy\anaconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [20]:
print("훈련 시간: {:.2f}s".format(t1 - t0))

훈련 시간: 71.88s


In [21]:
y_pred = log_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9255

**3.6 문제 : 3.2에서 축소된 데이터 셋을 사용해 소프트맥스 회귀모델을 훈련시켜보세요. 그리고 훈련시간을 확인하세요**

In [13]:

from sklearn.linear_model import LogisticRegression
log_clf = LogisticRegression(multi_class="multinomial", solver="lbfgs", random_state=42)
t0 = time.time()
log_clf.fit(X_train, y_train)
t1 = time.time()

C:\Users\jeehy\anaconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [16]:
t1-t0

80.43239116668701

훈련시간이 굉장히 빨라짐

**3.7 문제 : 모델의 정확도도 확인해보세요**

In [14]:
y_pred = log_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9255

**3.8 문제 : 랜덤포레스트 모형의 훈련시간과 정확도, 소프트맥스 모형의 훈련시간과 정확도를 비교해보세요.**

랜덤포레스트의 정확도는 0.9705, 소프트맥스 모형의 정확도는 0.9255로, 소프트맥스 모형의 정확도가 높다.
소프트맥스 모형의 훈련시간이 랜덤포레스트의 2배정도 빠르다. 